In [1]:
import os
import sys 
import pandas as pd 

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.animation as animation

In [7]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [9]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [10]:
import warnings  
warnings.filterwarnings('ignore')

2. 주제

- 포스트 코로나 시대 유망 품목 및 산업 발굴

- 코로나로 인한 국민의 생활/소비의 변화 파악

In [2]:
file_list = os.listdir('./KT_data')
file_list

['adstrd_master.csv',
 'card.csv',
 'Case.csv',
 'COVID_19',
 'COVID_eng_kor_table.xlsx',
 'Data-Science-for-COVID-19',
 'delivery.csv',
 'fpopl.csv',
 'index.csv',
 'Patient',
 'Policy.csv',
 'Region.csv',
 'SearchTrend.csv',
 'SeoulFloating.csv',
 'Time',
 'Weather.csv']

In [11]:
direc = "./KT_data/"

fpopl_columns = pd.read_csv(direc + "fpopl.csv", nrows = 1) # 행정동 유동인구

dtype = dict(zip(list(fpopl_columns.columns), ['object'] * 5 + ['int']))

fpopl = pd.read_csv(direc + "fpopl.csv", dtype = dtype) # 행정동 유동인구

fpopl.head() # 24시간 구분, 성별, 연령, 시군구, 유동인구

In [36]:
fpopl.base_ymd.min(), fpopl.base_ymd.max() 

fpopl.popltn_cascnt.describe()

('20200101', '20200616')

In [22]:
fpopl.query("base_ymd == '20200109'").adstrd_code.str.slice(0,2).unique() # 서울만

array(['11'], dtype=object)

In [26]:
fpopl.query("base_ymd == '20200109' \
            or base_ymd == '20200209' \
            or base_ymd == '20200309'").groupby(["base_ymd", fpopl.adstrd_code.str.slice(0,2) ]).agg({'popltn_cascnt': 'sum'})

,,popltn_cascnt
base_ymd,adstrd_code,
20200109,11,58779586
20200209,11,42685609
20200309,11,51249534


## 유동인구 수 변화

1. 코로나 이전과 이후의 유동인구 수 변화, 지역별 변화 확인
2. 남, 여 차이 존재하는지 
3. 연령별 차이 존재하는지 (14살이하,  1524 , 2534, 3549, 5059, 6069, 70대 이상)
4. 하루 평균 유동인구 나타내기 , 9시 ~ 12시 
결론: 국민의 생활 -> 유동인구가 줄어들고 거리두기를 지속하는 형태가 장기화 될 수 있음을 시사. 


https://dailyheumsi.tistory.com/118#2.-plotly-express-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0

## 코로나 확진자와 유동인구수의 관계
1. 코로나 확진자와 유동인구수 상관관계 조사. 
2. 코로나 사태 진정과 재확산의 기조의 증상과 현상

결론 : 이제 코로나 시대에서는 '' 할 것으로 보임. 

In [32]:
# 하루 평균 유동인구 수 비교 
groupby_ymd = fpopl.groupby('base_ymd')

In [42]:
import cufflinks as cf 
cf.go_offline(connected=True)

In [59]:
daily_ = groupby_ymd.agg({'popltn_cascnt': 'mean'})

In [60]:
daily_.index = pd.to_datetime(daily_.index)

In [61]:
daily_.iplot(kind='bar')

In [109]:
fpopl.dtypes

base_ymd           object
tmzon_se_code      object
sexdstn_se_code    object
agrde_se_code      object
adstrd_code        object
popltn_cascnt       int32
dtype: object

In [64]:
pivot = fpopl.pivot_table(values = 'popltn_cascnt', index = 'base_ymd',
                          columns =  fpopl.adstrd_code.str.slice(0,5), aggfunc = sum)

In [65]:
pivot

adstrd_code,11110,11140,11170,11200,11215,11230,11260,11290,11305,11320,...,11500,11530,11545,11560,11590,11620,11650,11680,11710,11740
base_ymd,,,,,,,,,,,,,,,,,,,,,
20200101,1289867,1250912,2122425,1892216,1832597,1400036,1270080,1768574,945218,1082708,...,2191177,1599510,911753,2307829,1580581,1694584,2984434,3028808,3251747,2094980
20200102,1883301,2151383,2461884,2316578,2069989,1757169,1410125,1909038,1010754,1186822,...,2470986,2158246,1192132,3080233,1846694,1938852,4118904,4823578,3793720,2315844
20200103,2008281,2279889,2659078,2469825,2223853,1825844,1476047,1983841,1058986,1235908,...,2593692,2261995,1256606,3264911,1956568,2055959,4431963,5192931,4068440,2461916
20200104,1685580,1669677,2533511,2287973,2188255,1685374,1454416,1941641,1036862,1214028,...,2465692,2017911,1067463,2858104,1879737,1977717,3979889,4268654,3863123,2425402
20200105,1342674,1345159,2224623,1938561,1870218,1464951,1282885,1766629,939133,1106379,...,2168499,1745106,908074,2469266,1628013,1748973,3312670,3409693,3307299,2170137
20200106,1888963,2244363,2481539,2377202,2106252,1773656,1405310,1944507,1018556,1196177,...,2508887,2208732,1213908,3154989,1900742,1975404,4218362,4935450,3860879,2346405
20200107,1838624,2189573,2435180,2329923,2045281,1735264,1387669,1893372,980671,1143321,...,2452203,2143256,1172897,3082539,1857391,1934423,4124040,4885801,3749944,2263520
20200108,1970437,2321619,2550428,2439448,2175464,1832477,1451387,1970056,1042603,1212970,...,2534048,2234019,1211257,3180717,1937172,2019871,4287482,5147585,4118888,2456357
20200109,2021552,2378019,2630836,2469053,2179853,1854431,1470359,1999770,1045560,1229273,...,2559187,2238902,1225272,3244857,1959767,2055911,4405553,5172761,4012419,2401547


In [67]:
adstrd_master = pd.read_csv('./KT_data/adstrd_master.csv', dtype = {'adstrd_code' : 'int'}) # 8자리 행정동 코드 데이터

In [77]:
adstrd_master.adstrd_code = adstrd_master.adstrd_code.astype("object")

In [78]:
adstrd_master.head()

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515,청운효자동,서울특별시,종로구
1,11110530,사직동,서울특별시,종로구
2,11110540,삼청동,서울특별시,종로구
3,11110550,부암동,서울특별시,종로구
4,11110560,평창동,서울특별시,종로구


In [124]:
gungu = adstrd_master.loc[:,['brtc_nm', 'signgu_nm', 'adstrd_code']]

gungu.dtypes

gungu.adstrd_code = gungu.adstrd_code.apply(lambda x : str(x)[:5])

seoul = gungu.drop_duplicates().query("brtc_nm == '서울특별시'").reset_index(drop=True)

In [136]:
seoul

,brtc_nm,signgu_nm,adstrd_code
0,서울특별시,종로구,11110
1,서울특별시,중구,11140
2,서울특별시,용산구,11170
3,서울특별시,성동구,11200
4,서울특별시,광진구,11215
5,서울특별시,동대문구,11230
6,서울특별시,중랑구,11260
7,서울특별시,성북구,11290
8,서울특별시,강북구,11305
9,서울특별시,도봉구,11320


In [139]:
pivot.index = pd.to_datetime(pivot.index)

In [140]:
pivot.iplot("line")

In [147]:
# http://opengov.seoul.go.kr/analysis/11143108
pivot[['11680', '11140']].iplot("line")

In [150]:
pivot.rolling(window = 7).mean()[['11680', '11140']].iplot("line")

In [ ]:
pd.DataFrame(fpopl.popltn_cascnt.describe(), dtype = 'int')

In [ ]:
# 1. 유동인구 데이터 분석

In [63]:
card = pd.read_csv("./KT_data/card.csv")

In [64]:
card.head()

,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
0,20200104,1174066000,성내3동,7041,약국,463,5843230
1,20200104,1174066000,성내3동,7022,치과의원,33,7835550
2,20200104,1174066000,성내3동,7021,한의원,53,4589800
3,20200104,1174066000,성내3동,7020,의원,339,9267240
4,20200104,1174066000,성내3동,6110,자동차정비,19,4441000


In [65]:
card.query("selng_cascnt != '석재' and salamt != '석재'", inplace = True)

In [70]:
for column in card.columns:
    if column in ['selng_cascnt', 'salamt'] :
        card[[column]] = card[[column]].astype("int")
    else :
        card[[column]] = card[[column]].astype("object")

ValueError: invalid literal for int() with base 10: '영상물'

In [68]:
card.head()

,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
0,20200104,1174066000,성내3동,7041,약국,463,5843230
1,20200104,1174066000,성내3동,7022,치과의원,33,7835550
2,20200104,1174066000,성내3동,7021,한의원,53,4589800
3,20200104,1174066000,성내3동,7020,의원,339,9267240
4,20200104,1174066000,성내3동,6110,자동차정비,19,4441000


In [54]:
card = pd.read_csv("./KT_data/card.csv", dtype = dtype)

ValueError: invalid literal for int() with base 10: '석재'

In [41]:
pd.DataFrame(card.describe(), dtype = 'int')

,receipt_dttm,adstrd_code,mrhst_induty_cl_code
count,3777647,3777647,3777647
mean,20200377,1144511491,6077
std,146,19188646,2387
min,20200104,1111051500,1001
25%,20200229,1126069000,4201
50%,20200406,1147054000,7020
75%,20200512,1162063000,8021
max,20200614,1174070000,9998


## 카드데이터 분석

1. 코로나 이전과 이후 가장 많이 차이가 나는 빈도 수 분석, 매출 분석
2. 코로나 이후 급상승한 트렌드 분석 
3. 

In [ ]:
card.

In [44]:
card.adstrd_code.str.slice(start=0, stop=5).unique()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [6]:
card.head()

,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
0,20200104,1174066000,성내3동,7041,약국,463,5843230
1,20200104,1174066000,성내3동,7022,치과의원,33,7835550
2,20200104,1174066000,성내3동,7021,한의원,53,4589800
3,20200104,1174066000,성내3동,7020,의원,339,9267240
4,20200104,1174066000,성내3동,6110,자동차정비,19,4441000


In [ ]:
card.groupby('selng_cascnt').salamt.sum().reset_index().sort_values(ascending=False)

In [ ]:
delivery = pd.read_csv("./KT_data/delivery.csv")

In [ ]:
covid_eng = pd.read_excel("./KT_data/COVID_eng_kor_table.xlsx")

In [ ]:
df = './KT_data/COVID_19/' 
fu = {}
for direc in os.listdir('./KT_data/COVID_19'):
    fu[direc.split("/")[-1].replace(".csv",'')] = df+direc

In [ ]:
for name, direc in fu.items():
    globals()['{}'.format(name.lower())] = pd.read_csv(direc)

In [ ]:
case.head() # group: 집단 감염 유무 

In [ ]:
for i in case.columns:
    print(i)
    print(case.loc[:,i].nunique(),'가지 category속성')
    print(case.loc[:,i].unique())
    print("\n")

In [ ]:
case.groupby("group").nunique()

In [ ]:
case.query("group == True").sort_values("confirmed", ascending = False).head(10)

In [ ]:
patientinfo.head()

In [ ]:
policy.head()

In [ ]:
region.head()

In [ ]:
time.head()

In [ ]:
timeage.head()

In [ ]:
timegender.head()

In [ ]:
timeprovince.head()